# Wrangle Data

In [ ]:
import pandas as pd
import datetime
import sqlite3
import zipfile
from sklearn import preprocessing

### Import Data

Crime Data:
- ZIP File.

In [ ]:
crime_zip = zipfile.ZipFile('../data/dc-crime-data/dc-crime-data.csv.zip', mode='r')
crime_csv = crime_zip.open('dc-crime-data.csv')
crime_df = pd.read_csv(crime_csv)

Weather Data
- DB Name: weather_data
- Table Name: weather_data

In [ ]:
conn = sqlite3.connect('../data/weather-data/weather_data.db')
weather_df = pd.read_sql('''select * from weather_data''', conn)
conn.close()

Census Data

- DB Name: census_bg
- Table Name: census_blockgroup

In [ ]:
conn = sqlite3.connect('../data/census-data/census_bg.db')
census_df = pd.read_sql('''select * from census_blockgroup''', conn)
conn.close()

### Wrangle Data

Crime Data:
- Create year, month, day, and time of day variables. TOD staddles mid-night, means interpretation of TOD is not continuous for the Midnight category.

In [ ]:
def assignTod(row):
    '''
    Assign time of day (TOD) based on the hour.
    '''
    try:
        timestamp = pd.Timestamp(row['START_DATE'])

        year = timestamp.year
        month = timestamp.month
        day = timestamp.day
        start_hour = timestamp.time().hour

        if 23 <= start_hour:
            tod_cat, tod_num = 'Midnight', 8
        if 0 <= start_hour < 2:
            tod_cat, tod_num = 'Midnight', 8
        elif 2 <= start_hour < 5:
            tod_cat, tod_num = 'Late Night', 1
        elif 5 <= start_hour < 8:
            tod_cat, tod_num = 'Early Morning', 2
        elif 8 <= start_hour < 11:
            tod_cat, tod_num = 'Morning', 3
        elif 11 <= start_hour < 14:
            tod_cat, tod_num = 'Afternoon', 4
        elif 14 <= start_hour < 17:
            tod_cat, tod_num = 'Mid Afternoon', 5
        elif 17 <= start_hour < 20:
            tod_cat, tod_num = 'Evening', 6
        elif 20 <= start_hour < 23:
            tod_cat, tod_num = 'Night', 7
    except:
        year, month, day, tod_cat, tod_num = '','','','',np.nan

    return year, month, day, tod_cat, tod_num

In [ ]:
crime_df[['year','month','day','tod_cat','tod_num']] = crime_df[['START_DATE']].apply(assignTod, axis=1, result_type='expand')

Weather Data:
- Remove negative time values, that is, time before 1970.
- Convert date-time to a format to match to the crime data, named "crime_time".
- Rename variables.
- Delete those that are non-numeric (one could encoding these variables, theoretically).

In [ ]:
def convertTime(row):
    '''
    Convert time elapsed from 1/1/1970 to YYYY-MM-DTHH:MM:SS.MMM format.
    '''
    time = datetime.datetime.fromtimestamp(row).strftime('%Y-%m-%dT%H:%M:%S.000')

    return time

In [ ]:
#There are negative time values that must be removed.
weather_df = weather_df[weather_df['currently_time'] > 0]

#Convert time to format shared by crime data.
weather_df['crime_time_format'] = weather_df['currently_time'].apply(convertTime)

#Rename and drop columns.
rename_cols = dict(currently_apparentTemperature = 'apparent_temp',
                   currently_cloudCover = 'cloud_cover',
                   currently_dewPoint = 'dew_point',
                   currently_humidity = 'humidity',
                   currently_icon = 'icon',
                   currently_precipIntensity = 'percip_intensity',
                   currently_precipProbability = 'percip_probability',
                   currently_precipType = 'percip_type',
                   currently_pressure = 'pressure',
                   currently_summary = 'summary',
                   currently_temperature = 'temperature',
                   currently_time = 'time',
                   currently_uvIndex = 'uv_index',
                   currently_visibility = 'visibility',
                   currently_windBearing = 'wind_bearing',
                   currently_windGust = 'wind_gust',
                   currently_windSpeed = 'wind_speed',
                   latitude = 'weather_latitude',
                   longitude = 'weather_longitude')
drop_cols = ['index','code','summary','icon','error','percip_type']

weather_df.rename(columns=rename_cols, inplace=True)
weather_df.drop(labels=drop_cols, axis='columns', inplace=True)

Census Data

In [ ]:
columns = ['UnWgtSampleCtPop','PerCapitaIncome','MedianHouseholdInc',
           'MedianAge','UnweightedSampleHousingUnits']

for col in columns:
    if census_df[col].dtypes == 'object':
        numeric_column = pd.to_numeric(census_df[col], errors = 'coerce')
        census_df[col] =  numeric_column

    mean = census_df[census_df[col] > 0][col].mean()
    census_df.fillna(value={col: mean}, inplace=True)
    census_df[census_df[col] < 0] = mean

census_df = census_df.loc[census_df['TotalPop'] > 0]

census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace(']]', '', regex=True)
census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace('\.0', '', regex=True)
census_df['Tract'] = census_df['Tract'].astype(str).replace('\.0', '', regex=True)
census_df['Tract'] = census_df['Tract'].apply(lambda x: x.zfill(6))
census_df['Year'] = census_df['Year'].astype(str).replace('\.0', '', regex=True)
census_df.rename(columns=dict(Year = 'census_year'), inplace=True)

census_df['index'] = census_df['Tract'] + census_df['BlockGroup'] + census_df['census_year']
census_df_nodup = census_df.drop_duplicates(subset='index')

### Merge Weather and Crime Data

- Merge weather and crime data before aggregating.

In [ ]:
crime_weather_mr = crime_df.merge(weather_df,
                                  how='left',
                                  left_on=['LATITUDE','LONGITUDE','START_DATE'],
                                  right_on=['weather_latitude','weather_longitude','crime_time_format'])

### Aggregate Crime/Weather Data

- Set "by_crime_type" to indicate whether to aggregate all crimes to by crime table (violent/non-violent).
- Count REPORT_DAT, since data is collected from MPD based on report date. This is like the most reliable variable to count instances of reported crime.
- For all variables except REPORT_DAT, we want to average. In this case, we average weather data across the TOD blocks.

In [ ]:
by_crime_type = False

In [ ]:
agg_vars = ['REPORT_DAT', 'apparent_temp', 'cloud_cover', 'dew_point', 'humidity',
            'percip_intensity', 'percip_probability', 'pressure', 'temperature', 'uv_index',
            'visibility', 'wind_bearing', 'wind_gust', 'wind_speed']

agg_dict = dict()

for var in agg_vars:
    if var == 'REPORT_DAT':
        agg_dict[var] = 'size'
    else:
        agg_dict[var] = 'mean'

if by_crime_type == False:
    crime_weather_agg = crime_weather_mr.groupby(by=['BLOCK_GROUP','year','month','day','tod_cat','tod_num'],
                                                 as_index=False).agg(agg_dict)
else:
    crime_weather_agg = crime_weather_mr.groupby(by=['BLOCK_GROUP','year','month','day','tod_cat','tod_num','offensegroup'],
                                                 as_index=False).agg(agg_dict)

rename_cols = dict(REPORT_DAT = 'crime_counts')
crime_weather_agg.rename(columns=rename_cols, inplace=True)

crime_weather_agg['index'] = crime_weather_agg['BLOCK_GROUP'] + crime_weather_agg['year'].astype('str')
crime_weather_agg['index'] = crime_weather_agg['index'].str.replace(" ","")

### Merge Weather-Crime and Census Data

In [ ]:
if by_crime_type == False:
    crime_weather_census = crime_weather_agg.merge(census_df, how='left', on='index')
else:
    crime_weather_census = crime_weather_agg.merge(census_df, how='left', on='index')

### Calculate and Standardize Crime Rates

- Calculate crime rates per 100,000 people.
- Remove instances of blank Census population data.
- Standardize crime rates before classification.

$$\text{Crime Rate}_{year, month, day, tod, bg, crime} = \frac{\text{Total Crime Count}_{year,month,day,tod,bg,crime}}{\text{Total Population}_{year,month,day,tod,bg,crime}} * 100,000$$

Calculate Crime Rates

In [ ]:
crime_weather_census_na = crime_weather_census.dropna(axis='index', how='any', subset=['TotalPop']).reset_index(drop=True)
crime_weather_census_na['crime_rate'] = (crime_weather_census_na['crime_counts'] / crime_weather_census_na['TotalPop'])*100000

Standardize Crime Rates

- Facilitate classification hypothesis by blocking crime rates.

In [ ]:
standardize = crime_weather_census_na[['crime_rate']]
power = preprocessing.PowerTransformer(method='box-cox', standardize=False)

In [ ]:
crime_weather_census_na['crs'] = power.fit_transform(standardize)

### Classify Crime Rates

Calc Stats

In [ ]:
stats = crime_weather_census_na[['crs']].describe().transpose()

Classify

In [ ]:
def classifyCrimeRates(row, stats):
    range_high = float(stats['mean'] + 2*stats['std'])
    range_low = float(stats['mean'] - 2*stats['std'])
    range_mid_high = float(stats['mean'] + stats['std'])
    range_mid_low = float(stats['mean'] - stats['std'])
    cr = float(row['crs'])
    
    if cr >= range_high:
        crime_rate_cat = 'High'
    if cr >= range_mid_high and cr < range_high:
        crime_rate_cat = 'Med-High'
    if cr >= range_mid_low and cr < range_mid_high:
        crime_rate_cat = 'Med'
    if cr >= range_low and cr < range_mid_low:
        crime_rate_cat = 'Low-Med'
    if cr < range_low:
        crime_rate_cat = 'Low'
    
    return crime_rate_cat

In [ ]:
crime_weather_census_na['crime_rate_cat'] = crime_weather_census_na[['crs']].apply(classifyCrimeRates,
                                                                                   args=(stats[['mean','std']],),
                                                                                   axis=1)

### Set Weekday Column

In [ ]:
def weekday(row):
    date = '{}-{}-{}'.format(str(row['year']), str(row['month']), str(row['day']))
    
    weekday = pd.Timestamp(date).weekday()
    
    return weekday

In [ ]:
crime_weather_census_na['weekday'] = crime_weather_census_na[['year', 'month', 'day']].apply(weekday, axis=1)

### Output

In [ ]:
table_name = 'all_crimes'

In [ ]:
conn = sqlite3.connect('')
c = conn.cursor()

c.execute("drop table if exists {}".format(table_name))

data.to_sql(table_name, conn)

conn.commit()
conn.close()